In [1]:
# Installer les dépendances nécessaires
%pip install -q langchain-google-genai pypdf langchain-chroma faiss-cpu sentence-transformers streamlit
%pip install langchain_community langchainhub chromadb langchain
%pip install -q python-dotenv # Pour gérer les variables d'environnement (API key)
%pip install -q transformers # Pour le modèle d'embedding HuggingFace

# Assurons-nous que LangChain est à jour pour les dernières fonctionnalités
%pip install -q --upgrade langchain

print("Toutes les dépendances ont été installées ou mises à jour.")

from google.colab import userdata
userdata.get('GOOGLE_API_KEY')
userdata.get('LANGCHAIN_API_KEY')


import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


Note: you may need to restart the kernel to use updated packages.
Toutes les dépendances ont été installées ou mises à jour.


ModuleNotFoundError: No module named 'google.colab'

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')
# # Path to the pdf file
# pdf_path = '/content/drive/MyDrive/Projet_AI31/Code_du_travail-23-300.pdf'

# Path to the pdf file
pdf_path = 'Code_du_travail-23-300.pdf'  # Assurez-vous que le fichier est dans le même dossier que le notebook


Mounted at /content/drive


# Objectif du projet
Mettre en place un système RAG (Retrieval Augmented Generation) dédié au Code du travail, servant d’**outil de requêtage intelligent** et d’**aide à la décision** pour des boîtes de **consultation juridique**.

L’objectif est de permettre aux utilisateurs (juristes, consultants, avocats juniors, etc.) de poser des questions en langage naturel et d’obtenir des réponses précises, contextualisées et appuyées par des sources juridiques fiables (textes de loi, jurisprudence, doctrine).

**NB :** le RAG qui sera mis sur pied sera spécifique au droit du travail. Ce serait très ambitieux de notre part de mettre sur pied un RAG couvrant l’ensemble du droit français étant donné le temps imparti pour la réalisation de ce projet (le droit étant vaste, technique et complexe).

# ETAPES CLES
1. **Préparation de l'environnement :** Installer les outils et librairies nécessaires.
2. **Chargement et découpage du document (Document Loading & Splitting) :**
Chargement du pdf de 2755 pages (code du travail). Un document aussi volumineux ne peut pas être traité d'un seul bloc par un modèle de langage. Nous devrons le découper en morceaux plus petits (appelés "chunks" ou "morceaux") tout en conservant le contexte. C'est crucial pour la pertinence des recherches.
3. **Création des embeddings (Embeddings Generation) :**
Chaque morceau de texte sera transformé en une représentation numérique appelée "embedding". C'est une sorte de "vecteur" qui capture la signification sémantique du texte. Ces embeddings nous permettront de trouver des morceaux de texte similaires à la question poser par un utilisateur (sous forme de requête).
3. **Stockage vectoriel avec Chroma (Vector Store with ChromaDB) :**
Nous allons stocker ces embeddings, ainsi que le texte original des morceaux, dans une base de données vectorielle appelée Chroma.
Chroma est optimisé pour les recherches de similarité vectorielle, ce qui est exactement ce dont nous avons besoin pour récupérer les informations pertinentes.
4. **Construction de la chaîne RAG avec LangChain (Building the RAG Chain with LangChain) :**
LangChain est un framework qui aide à orchestrer les différentes étapes (récupération de documents, génération de réponses).
Nous utiliserons LangChain pour relier notre base de données vectorielle (Chroma) et notre modèle de langage (Gemini).
5. **Intégration du modèle de langage Gemini (LLM Integration with Gemini) :**
Gemini sera le cerveau de notre application. Il prendra les morceaux de texte pertinents et la question pour générer une réponse cohérente et informative.
6. **Construction de l'interface utilisateur avec Streamlit (Streamlit UI) :**
Streamlit nous permettra de créer une interface web simple et interactive où l'utilisateur pourras poser ses questions et voir les réponses, y compris les sources.
7. **Gestion des sources/Gestion des Hallucinations (Source Retrieval) :**
Nous nous assurerons que chaque réponse est accompagnée de l'indication de la page ou du morceau de texte d'où provient l'information.

## Pourquoi ces outils ?

1. **LangChain :** C'est un framework puissant qui simplifie énormément la construction d'applications basées sur les grands modèles de langage (LLMs). Il offre des composants modulaires pour le chargement de documents, le découpage, les bases de données vectorielles, et l'intégration de LLMs.
2. **Chroma :** Une base de données vectorielle légère et facile à utiliser, parfaite pour les projets de taille moyenne et l'apprentissage. Elle est souvent utilisée avec LangChain.
3. **Gemini :** Un modèle de langage performant de Google, capable de comprendre le langage naturel et de générer des réponses pertinentes.
4. **Streamlit :** C'est un excellent outil pour construire rapidement des interfaces utilisateur pour des applications de machine learning et de data science, sans avoir besoin de connaissances approfondies en développement web.

## PARTIE INDEXATION


In [4]:
import re
from typing import List, Optional

from langchain.text_splitter import TextSplitter
from langchain.schema import Document

class TitleBasedSplitter(TextSplitter):
    """
    Splitter de texte basé sur la détection de titres.
    Garde les métadonnées de la page d'origine pour chaque chunk.
    """
    def __init__(self, pattern: str = r"(Titre\s+(?:[IVXLCDM]+(?:er|ème)?|[A-Za-z\d\u00C0-\u00FF'-]+)\s*:)"):
        """
        Initialise le splitter avec un pattern regex pour les titres.
        Le pattern par défaut cherche "Titre " suivi de:
        - Soit des chiffres romains (I, V, X...) optionnellement suivis de "er" ou "ème".
        - Soit des mots (lettres, chiffres, accents, apostrophes, tirets).
        - Le tout suivi d'un deux-points (avec espaces optionnels autour).
        Exemples: "Titre Ier :", "Titre préliminaire :", "Titre II :"
        """
        super().__init__()
        self.compiled_pattern = re.compile(pattern, re.IGNORECASE)

    def split_documents(self, documents: List[Document], **kwargs) -> List[Document]:
        """
        Divise une liste de documents (pages) en chunks basés sur les titres.
        Chaque chunk hérite des métadonnées de son document d'origine.
        """
        all_chunks: List[Document] = []
        for doc in documents:
            text = doc.page_content
            metadata = doc.metadata.copy() # Copie des métadonnées de la page d'origine

            matches = list(self.compiled_pattern.finditer(text))

            if not matches:
                # Si aucun titre n'est trouvé sur la page, la page entière est un chunk
                if text.strip(): # S'assurer que le texte n'est pas vide
                    all_chunks.append(Document(page_content=text.strip(), metadata=metadata))
                continue # Passer à la page suivante

            # Cas où il y a du texte avant le premier titre sur la page
            if matches[0].start() > 0:
                pre_title_text = text[0:matches[0].start()].strip()
                if pre_title_text:
                    all_chunks.append(Document(page_content=pre_title_text, metadata=metadata))

            for i in range(len(matches)):
                start = matches[i].start()
                # La fin du chunk est le début du prochain match, ou la fin du texte si c'est le dernier match
                end = matches[i+1].start() if i + 1 < len(matches) else len(text)

                chunk_content = text[start:end].strip()

                if chunk_content:
                    # Ajout de métadonnées spécifiques au chunk si besoin,
                    # par exemple le titre identifié ou une ID de chunk
                    chunk_metadata = metadata.copy()
                    # On peut ajouter ici des infos comme le titre spécifique du chunk
                    # chunk_metadata["title"] = matches[i].group(0).strip()
                    all_chunks.append(Document(page_content=chunk_content, metadata=chunk_metadata))

        return all_chunks

    # Nous n'avons pas besoin de split_text si nous utilisons split_documents directement sur des objets Document.
    # Cependant, si l'on veux sonserver la compatibilité avec d'autres TextSplitter,
    # l'on peut implémenter split_text pour une chaîne unique.
    def split_text(self, text: str) -> List[str]:
        """
        Divise une chaîne de texte en chunks basés sur les titres.
        Cette méthode est appelée par split_documents si un Document est passé.
        """
        chunks = []
        matches = list(self.compiled_pattern.finditer(text))

        if not matches:
            return [text.strip()] if text.strip() else []

        # Ajouter le texte avant le premier titre comme un chunk si non vide
        if matches[0].start() > 0:
            pre_title_text = text[0:matches[0].start()].strip()
            if pre_title_text:
                chunks.append(pre_title_text)

        for i in range(len(matches)):
            start = matches[i].start()
            end = matches[i+1].start() if i + 1 < len(matches) else len(text)
            chunk_content = text[start:end].strip()
            if chunk_content:
                chunks.append(chunk_content)
        return chunks

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document


if not os.path.exists(pdf_path):
    print(f"ERREUR: Le fichier PDF '{pdf_path}' n'a pas été trouvé. Veuillez vérifier le chemin.")
else:
    print(f"Chargement du PDF depuis : {pdf_path}")

    # 1. Charger les documents page par page
    loader = PyPDFLoader(pdf_path)
    docs_from_loader = loader.load() # Chaque élément est un Document (page) avec ses métadonnées

    if not docs_from_loader:
        print("Aucun document n'a pu être chargé par PyPDFLoader. Veuillez vérifier le PDF.")
        documents = [] # L'on s'assure que 'documents' est défini même si le chargement échoue
    else:
        print(f"Nombre de pages chargées par PyPDFLoader : {len(docs_from_loader)}")

        # 2. Initialiser notre splitter basé sur les titres
        splitter = TitleBasedSplitter()

        # 3. Appliquer le splitter sur chaque document (page)
        # La méthode split_documents prend une liste de Document et retourne une liste de Document.
        documents = splitter.split_documents(docs_from_loader)

        print(f"Nombre de documents (chunks) après TitleBasedSplitter : {len(documents)}")

        # Affichage de quelques exemples de chunks pour vérifier leur contenu et leurs métadonnées
        for i, doc_chunk in enumerate(documents[:5]): # Affiche les 5 premiers chunks
            print(f"\n--- Chunk {i+1} ---")
            print(f"Page content (début): {doc_chunk.page_content[:500]}...") # Affiche les 500 premiers caractères
            print(f"Metadata: {doc_chunk.metadata}")

            # Vérifier si 'page' est dans les métadonnées
            if 'page' in doc_chunk.metadata:
                print(f"Source Page: {doc_chunk.metadata['page'] + 1}") # +1 car les pages sont souvent 0-indexées

# Le code ou les fonctions des parties embeddings, vectorstore, LLM et chaîne RAG pourront suivre ici
# NB : Il est important de s'assurer qu'il n'y a pas d'erreur dans 'documents' si aucun document n'est chargé.
if not documents:
    print("Le traitement ne peut pas continuer car aucun document n'a été créé.")
    sys.exit()  # Ou raise une exception ici si l'absence de documents est critique.

Chargement du PDF depuis : /content/drive/MyDrive/Projet_AI31/Code_du_travail-23-300.pdf
Nombre de pages chargées par PyPDFLoader : 278
Nombre de documents (chunks) après TitleBasedSplitter : 312

--- Chunk 1 ---
Page content (début): Partie législative - Chapitre préliminaire : Dialogue social. 
Partie législative
Chapitre préliminaire : Dialogue social.
L. 1  LOI n°2008-67 du 21 janvier 2008 - art. 3      
  Legif.   
  Plan   
  Jp.Judi.   
  Jp.Admin.   
  Juricaf  
Tout projet de réforme envisagé par le Gouvernement qui porte sur les relations individuelles et collectives
du travail, l'emploi et la formation professionnelle et qui relève du champ de la négociation nationale et
interprofessionnelle fait l'objet d'une conc...
Metadata: {'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-06-03T20:04:09+00:00', 'source': '/content/drive/MyDrive/Projet_AI31/Code_du_travail-23-300.pdf', 'total_pages': 278, 'page': 0, 'page_label': '1'}
Source Page: 1

--- C

In [6]:
import re
from typing import List, Optional, Dict, Any

from langchain.text_splitter import TextSplitter
from langchain.schema import Document

class CodeDuTravailStructureExtractor(TextSplitter):
    """
    Splitter et extracteur de structure pour le Code du Travail.
    Découpe le document en chunks et enrichit les métadonnées
    avec les informations de Partie, Livre, Titre, Chapitre, Section et Article.
    """
    def __init__(
        self,
        # Pattern pour les titres principaux (Titre Ier :, Titre Préliminaire :)
        title_pattern: str = r"(Titre\s+(?:[IVXLCDM]+(?:er|ème)?|[A-Za-z\d\u00C0-\u00FF'-]+)\s*:\s*.+?)(?=\n(?:Titre\s|Chapitre\s|Section\s|Article\s|$))",
        # Pattern pour les chapitres (Chapitre Ier :, Chapitre Unique :)
        chapter_pattern: str = r"(Chapitre\s+(?:[IVXLCDM]+(?:er|ème)?|unique|[A-Za-z\d\u00C0-\u00FF'-]+)\s*:\s*.+?)(?=\n(?:Titre\s|Chapitre\s|Section\s|Article\s|$))",
        # Pattern pour les sections (Section 1 :, Section unique :)
        section_pattern: str = r"(Section\s+(?:\d+|unique|[A-Za-z\d\u00C0-\u00FF'-]+)\s*:\s*.+?)(?=\n(?:Titre\s|Chapitre\s|Section\s|Article\s|$))",
        # Pattern pour les articles (L. 123-1, R. 456-7, D. 789-10)
        article_pattern: str = r"(Article\s+((?:L|R|D)\.\s*\d{3}-\d+(?:-\d+)?(?:-\d+)?)[\s\S]*?(?=Article\s+((?:L|R|D)\.\s*\d{3}-\d+(?:-\d+)?(?:-\d+)?)|Titre\s+|Chapitre\s+|Section\s+|$))",
        # Capture le numéro de l'article dans un groupe pour extraction facile
        article_num_capture_group: int = 2, # Le groupe qui contient "L. 123-1"
        keep_separator: bool = True, # Indique si le séparateur (titre/chapitre/article) doit faire partie du chunk
        **kwargs
    ):
        super().__init__(keep_separator=keep_separator, **kwargs)
        self.title_pattern = re.compile(title_pattern, re.IGNORECASE | re.DOTALL)
        self.chapter_pattern = re.compile(chapter_pattern, re.IGNORECASE | re.DOTALL)
        self.section_pattern = re.compile(section_pattern, re.IGNORECASE | re.DOTALL)
        self.article_pattern = re.compile(article_pattern, re.IGNORECASE | re.DOTALL)
        self.article_num_capture_group = article_num_capture_group

    def split_documents(self, documents: List[Document]) -> List[Document]:
        """
        Divise les documents (pages) en chunks et enrichit les métadonnées.
        """
        all_chunks: List[Document] = []
        current_book = None
        current_part = None
        current_title = None
        current_chapter = None
        current_section = None

        # Pour le Code du Travail, la "Partie législative" et "Partie réglementaire"
        # sont souvent des sections de haut niveau. On pourrait les détecter aussi.
        # Par exemple: r"(Partie\s+(?:législative|réglementaire)\s*:\s*.+?)(?=\n(?:Partie\s|Livre\s|Titre\s|Chapitre\s|Section\s|Article\s|$))"

        for doc in documents:
            text = doc.page_content
            page_metadata = doc.metadata.copy() # Copier les métadonnées de la page (ex: page number)

            # Il est crucial de détecter les éléments hiérarchiques dans l'ordre décroissant
            # (Partie -> Livre -> Titre -> Chapitre -> Section -> Article)
            # pour s'assurer que les métadonnées sont correctement propagées.

            # Nous allons traiter le texte de la page pour identifier les blocs.
            # L'idée est de trouver tous les matchs de titres, chapitres, sections, articles
            # et de les traiter séquentiellement.

            # Simple approche pour commencer: On va splitter principalement par article
            # et ensuite extraire les informations de titre/chapitre/section
            # qui précèdent cet article ou sont détectées dans le chunk de l'article.

            # Cette méthode nécessite une logique plus complexe pour propager le contexte
            # (quel titre/chapitre actuel s'applique à un article).
            # On va utiliser une approche par "contexte courant" qui est mise à jour à chaque détection.

            # Initialisation du contexte pour cette page (on garde le dernier contexte connu)
            # Pour la première page, ces valeurs seront None.
            # Pour les pages suivantes, elles hériteront des valeurs de la fin de la page précédente.

            # TODO: Implémenter la logique de propagation du contexte inter-pages
            # Pour l'instant, on suppose que les titres sont répétés ou que les chunks sont suffisamment petits
            # pour qu'un article soit dans son contexte de titre/chapitre.
            # C'est un point clé à affiner si les sources ne sont pas précises.

            # Découpage par Article principalement
            article_matches = list(self.article_pattern.finditer(text))

            last_idx = 0
            # Si du texte précède le premier article
            if article_matches and article_matches[0].start() > 0:
                pre_article_text = text[0:article_matches[0].start()].strip()
                if pre_article_text:
                    # Tentative d'extraction des infos de titre/chapitre/section pour ce bloc introductif
                    # C'est une simplification, idéalement on détecterait ces éléments en amont du split.
                    chunk_metadata = self._extract_hierarchy_metadata(pre_article_text, page_metadata)
                    all_chunks.append(Document(page_content=pre_article_text, metadata=chunk_metadata))
                last_idx = article_matches[0].start()

            for i, match in enumerate(article_matches):
                article_content = match.group(0).strip() # Le contenu complet de l'article (incluant le numéro)
                article_number = match.group(self.article_num_capture_group) # Le numéro de l'article

                # Ici, nous pourrions affiner l'extraction de la date ou de la loi spécifique
                # si un pattern est détectable dans l'article_content
                # Par exemple: LOI n° 2014-288 du 5 mars 2014-art. 29 (V)
                # pattern_date_loi = r"(LOI n°\s*\d{4}-\d+\s*du\s+\d{1,2}\s+\w+\s+\d{4})"
                # date_loi_match = re.search(pattern_date_loi, article_content)
                # if date_loi_match:
                #     article_metadata["date_loi"] = date_loi_match.group(1)

                chunk_metadata = page_metadata.copy()
                chunk_metadata["type"] = "Article"
                chunk_metadata["article_number"] = article_number

                # Tenter d'extraire le contexte hiérarchique pour cet article
                # On recherche les titres, chapitres, sections qui précèdent immédiatement cet article
                # dans la portion de texte traitée pour cette page.

                # C'est une simplification pour l'exemple.
                # Une vraie solution robuste impliquerait de parser le document séquentiellement
                # et de maintenir un état des "titres actifs" et "chapitres actifs"
                # au fur et à mesure que l'on lit le document.

                # Pour l'instant, on fait une extraction locale:
                chunk_metadata.update(self._extract_hierarchy_metadata(article_content, page_metadata))

                all_chunks.append(Document(page_content=article_content, metadata=chunk_metadata))
                last_idx = match.end()

            # Si du texte reste après le dernier article sur la page
            if last_idx < len(text):
                remaining_text = text[last_idx:].strip()
                if remaining_text:
                    chunk_metadata = self._extract_hierarchy_metadata(remaining_text, page_metadata)
                    all_chunks.append(Document(page_content=remaining_text, metadata=chunk_metadata))

        return all_chunks

    def _extract_hierarchy_metadata(self, text: str, base_metadata: Dict[str, Any]) -> Dict[str, Any]:
        """
        Tente d'extraire les informations de hiérarchie (Titre, Chapitre, Section)
        à partir d'un morceau de texte et les ajoute aux métadonnées.
        """
        metadata = base_metadata.copy()

        # Chercher le titre le plus proche
        title_match = self.title_pattern.search(text)
        if title_match:
            metadata["title"] = title_match.group(1).strip()
            # On peut aussi essayer de parser le numéro et le nom si le format est fixe
            # Ex: "Titre Ier : Dispositions générales" -> {"title_num": "Ier", "title_name": "Dispositions générales"}

        # Chercher le chapitre le plus proche
        chapter_match = self.chapter_pattern.search(text)
        if chapter_match:
            metadata["chapter"] = chapter_match.group(1).strip()
            # Ex: "Chapitre Ier : Objet et portée" -> {"chapter_num": "Ier", "chapter_name": "Objet et portée"}

        # Chercher la section la plus proche
        section_match = self.section_pattern.search(text)
        if section_match:
            metadata["section"] = section_match.group(1).strip()
            # Ex: "Section 1 : Définitions" -> {"section_num": "1", "section_name": "Définitions"}

        # Pour les livres et parties, c'est plus complexe car ils sont souvent en début de document
        # ou indiqués par des en-têtes/pieds de page non extraits par `page_content`.
        # Pour le "Livre" et la "Partie", il est souvent plus efficace de les extraire
        # soit manuellement au début, soit en faisant une passe initiale sur le document complet
        # pour établir la hiérarchie.
        # Pour l'instant, je ne les inclus pas dans _extract_hierarchy_metadata,
        # car un chunk d'article ne les contiendra pas nécessairement.
        # On pourrait les ajouter comme des métadonnées globales au document si elles sont fixes,
        # ou les propager depuis un parseur de document entier.

        return metadata

    # La méthode split_text n'est pas utilisée directement par split_documents,
    # mais je la garde pour la conformité avec la classe mère, même si elle n'est pas optimisée
    # pour notre usage actuel d'extraction de métadonnées.
    def split_text(self, text: str) -> List[str]:
        # Cette implémentation est plus simple, mais ne gère pas la propagation des métadonnées
        # C'est pourquoi nous utiliserons principalement split_documents.
        chunks = []
        # On va splitter par les articles pour avoir des chunks plus petits
        article_splits = self.article_pattern.split(text)

        # Si le premier élément n'est pas un séparateur, c'est un préambule
        if len(article_splits) > 0 and not self.article_pattern.match(article_splits[0]):
            if article_splits[0].strip():
                chunks.append(article_splits[0].strip())
            start_index = 1
        else:
            start_index = 0

        # Réassembler les articles avec leur numéro
        for i in range(start_index, len(article_splits), self.article_num_capture_group + 1):
            if i + self.article_num_capture_group < len(article_splits):
                article_num = article_splits[i + self.article_num_capture_group -1] # Le numéro de l'article capturé
                article_content = article_splits[i + self.article_num_capture_group] # Le contenu après le numéro

                full_article_chunk = f"Article {article_num.strip()} {article_content.strip()}"
                if full_article_chunk.strip():
                    chunks.append(full_article_chunk.strip())
            elif article_splits[i].strip():
                chunks.append(article_splits[i].strip())
        return chunks

In [7]:
import sys

if not os.path.exists(pdf_path):
    print(f"ERREUR: Le fichier PDF '{pdf_path}' n'a pas été trouvé. Veuillez vérifier le chemin.")
    documents = [] # Assure-toi que 'documents' est défini même si le fichier est manquant
else:
    print(f"Chargement du PDF depuis : {pdf_path}")

    loader = PyPDFLoader(pdf_path)
    # Chargement page par page pour conserver les métadonnées de page
    docs_from_loader = loader.load()

    if not docs_from_loader:
        print("Aucun document n'a pu être chargé par PyPDFLoader. Veuillez vérifier le PDF.")
        documents = []
    else:
        print(f"Nombre de pages chargées par PyPDFLoader : {len(docs_from_loader)}")

        # Initialiser notre extracteur de structure
        # On va créer des patterns plus robustes si le test initial ne donne pas satisfaction
        # Les patterns par défaut dans la classe sont un bon point de départ.
        structure_extractor = CodeDuTravailStructureExtractor()

        # Appliquer l'extracteur sur les documents (pages)
        documents = structure_extractor.split_documents(docs_from_loader)

        print(f"Nombre de documents (chunks enrichis) après extraction de structure : {len(documents)}")

        # Afficher quelques exemples de chunks enrichis
        for i, doc_chunk in enumerate(documents[:5]): # Affiche les 5 premiers chunks
            print(f"\n--- Chunk {i+1} ---")
            print(f"Page content (début): {doc_chunk.page_content[:500]}...")
            print(f"Metadata: {doc_chunk.metadata}")
            # Exemple de vérification des métadonnées spécifiques
            if 'page' in doc_chunk.metadata:
                print(f"  Source Page: {doc_chunk.metadata['page'] + 1}")
            if 'type' in doc_chunk.metadata:
                print(f"  Type de chunk: {doc_chunk.metadata['type']}")
            if 'article_number' in doc_chunk.metadata:
                print(f"  Numéro d'Article: {doc_chunk.metadata['article_number']}")
            if 'title' in doc_chunk.metadata:
                print(f"  Titre: {doc_chunk.metadata['title']}")
            if 'chapter' in doc_chunk.metadata:
                print(f"  Chapitre: {doc_chunk.metadata['chapter']}")

# Assurez-vous que 'documents' est défini pour les étapes suivantes même en cas d'échec de chargement
if not documents:
    print("Le traitement ne peut pas continuer car aucun document structuré n'a été créé.")
    sys.exit()  # Ou raise une exception ici si l'absence de documents est critique.

Chargement du PDF depuis : /content/drive/MyDrive/Projet_AI31/Code_du_travail-23-300.pdf
Nombre de pages chargées par PyPDFLoader : 278
Nombre de documents (chunks enrichis) après extraction de structure : 411

--- Chunk 1 ---
Page content (début): Partie législative - Chapitre préliminaire : Dialogue social. 
Partie législative
Chapitre préliminaire : Dialogue social.
L. 1  LOI n°2008-67 du 21 janvier 2008 - art. 3      
  Legif.   
  Plan   
  Jp.Judi.   
  Jp.Admin.   
  Juricaf  
Tout projet de réforme envisagé par le Gouvernement qui porte sur les relations individuelles et collectives
du travail, l'emploi et la formation professionnelle et qui relève du champ de la négociation nationale et
interprofessionnelle fait l'objet d'une conc...
Metadata: {'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-06-03T20:04:09+00:00', 'source': '/content/drive/MyDrive/Projet_AI31/Code_du_travail-23-300.pdf', 'total_pages': 278, 'page': 0, 'page_label': '1', 'chapte

## PARTIES EMBEDDING, STOCKAGE VECTORIELLE, CHAINE RAG, LLM

### Pourquoi découper le document (Chunking) ?

- **Limitation des LLM :** Les grands modèles de langage ont une "fenêtre de contexte" limitée. C'est la quantité de texte qu'ils peuvent lire et traiter en une seule fois. Un PDF de 2755 pages est bien au-delà de ce qu'ils peuvent gérer.
- **Pertinence de la recherche :** Si l'on cherche un concept dans 2775 pages, la recherche serait très lente et peu précise.
_Solution :_
**Le découpage (Chunking)** - Nous allons diviser le PDF en petits morceaux (des paragraphes, quelques phrases, etc.). Chaque morceau est suffisamment petit pour être traité par un LLM et suffisamment grand pour conserver du sens.
_Conséquence :_ Quand un utilisateur posera une question, les morceaux les plus pertinenets seront chercher et non l'intégralité du document.


### Qu'est-ce qu'un Embedding et pourquoi en a-t-on besoin ?
Imaginons que chaque mot, phrase ou morceau de texte peut être représenté par un point dans un espace multidimensionnel (un peu comme des coordonnées X, Y, Z, mais avec beaucoup plus de dimensions).

- **Transformation en nombres :** Un "embedding" est une suite de nombres (un vecteur) qui représente la signification sémantique d'un texte. Des textes qui ont un sens similaire seront représentés par des vecteurs "proches" dans cet espace.
- **Recherche de similarité :** La question de l'utilisateur sera transformée en embedding. Ensuite, une recherche des embeddings de morceaux de texte qui sont les plus "proches" (on pourra notamment faire usage de l'algorithme des K-plus proches voisins pour récupérer les informations les plus pertinentes) de l'embedding da la question se fera dans la base de données. C'est la base de la recherche sémantique.

_Pourquoi ?_ Un ordinateur ne "comprend" pas le texte. Il comprend les nombres. Les embeddings sont le pont entre le langage humain et la capacité de l'ordinateur à trouver des similarités de sens.

### Le rôle de la base de données vectorielle (ChromaDB)
Une fois que nous avons nos morceaux de texte et leurs embeddings, où les stockons-nous de manière efficace pour pouvoir les rechercher rapidement ?

- **Base de données traditionnelle vs. Base de données vectorielle :** Une base de données classique (comme SQL) est optimisée pour des recherches exactes (ex: "trouve tous les utilisateurs dont le nom est 'Dupont'"). Une base de données vectorielle est optimisée pour des recherches de similarité (ex: "trouve tous les textes qui sont similaires à 'licenciement abusif'").

**ChromaDB :** C'est une base de données vectorielle légère et facile à mettre en place. Elle va stocker :
- Les morceaux de texte bruts (le contenu original).
- Les embeddings correspondants (les vecteurs numériques).
- Des métadonnées (comme le numéro de page d'où vient le morceau, ce qui est crucial pour les sources !).
_Fonctionnement :_ Quand l'utilisateur pose une question, Chroma va rapidement identifier les embeddings les plus similaires à sa question, et lui retourner les morceaux de texte correspondants.


### LangChain : L'orchestrateur de notre application
LangChain est un framework puissant qui simplifie énormément la construction d'applications basées sur les grands modèles de langage.

_Concept de "Chains" (Chaînes) :_ LangChain permet de relier différents composants (comme le chargement de documents, les découpeurs, les modèles d'embeddings, les bases de données vectorielles, et les LLM) dans une séquence logique, une "chaîne".

- **Modularité :** Chaque partie de l'application RAG (chargement, découpage, embedding, recherche, génération) est un "maillon" de la chaîne. LangChain permet de les assembler facilement.
- **Simplification :** Au lieu de gérer manuellement chaque interaction entre ces composants, LangChain fournit des abstractions qui rendent le développement plus rapide et plus propre.


### Gemini : Le cerveau qui génère la réponse
Gemini est le modèle de langage (LLM) de Google que nous allons utiliser.

_Son rôle :_ Une fois que Chroma a récupéré les morceaux de texte les plus pertinents du document/PDF, Gemini va recevoir :
- La question.
- Les morceaux de texte pertinents.

_Sa tâche :_ Il va analyser ces informations et générer une réponse cohérente, synthétisée, et basée sur le contexte fourni. C'est là que la "Génération" de RAG entre en jeu.
- **API :** Nous interagirons avec Gemini via son API (Interface de Programmation d'Application), ce qui signifie que nous enverrons des requêtes à un service Google pour obtenir des réponses.

### Streamlit : L'interface utilisateur simple et rapide
Pour interagir avec notre application, nous avons besoin d'une interface.

####Qu'est-ce que c'est ?
**Streamlit** est une bibliothèque Python qui permet de créer très facilement des applications web interactives pour la science des données et le machine learning.
_Avantages :_ Pas besoin d'être un expert en développement web (HTML, CSS, JavaScript). Avec quelques lignes de Python, tu peux créer des widgets (champs de texte, boutons) et afficher des résultats.

_Notre utilisation :_ Nous l'utiliserons pour créer une boîte de texte où l'utilisateur pourras taper ses questions, un bouton pour les soumettre, et un espace pour afficher la réponse de Gemini et les sources.

In [13]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain import hub
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda


# S'assurer que 'documents' n'est pas vide avant de continuer
if not documents:
    print("ATTENTION : La liste 'documents' est vide. Le processus RAG ne peut pas continuer.")
    exit("Processus arrêté car aucun document n'a été traité.")

print("\n--- Étape 4 : Création des Embeddings ---")
embedding_model_name = "all-MiniLM-L6-v2"
print(f"Initialisation du modèle d'embedding : {embedding_model_name}")

try:
    embedding_model = HuggingFaceEmbeddings(model_name=embedding_model_name)
    print("Modèle d'embedding initialisé avec succès.")
except Exception as e:
    print(f"ERREUR lors de l'initialisation du modèle d'embedding : {e}")
    print("Veuillez vérifier votre connexion internet et les dépendances 'sentence-transformers'.")
    exit("Processus arrêté car le modèle d'embedding n'a pas pu être initialisé.")


print("\n--- Étape 5 : Création ou chargement du Vectorstore Chroma ---")
chroma_db_dir = "./chroma_db_codetravail"

if os.path.exists(chroma_db_dir) and os.listdir(chroma_db_dir):
    print(f"Chargement du vectorstore Chroma existant depuis : {chroma_db_dir}")
    try:
        vectorstore = Chroma(persist_directory=chroma_db_dir, embedding_function=embedding_model)
        print("Vectorstore Chroma chargé.")
    except Exception as e:
        print(f"ERREUR lors du chargement du vectorstore Chroma : {e}")
        print("Tentative de recréation du vectorstore...")
        vectorstore = Chroma.from_documents(documents, embedding=embedding_model, persist_directory=chroma_db_dir)
        print("Vectorstore Chroma recréé.")
else:
    print(f"Création d'un nouveau vectorstore Chroma dans : {chroma_db_dir}")
    vectorstore = Chroma.from_documents(documents, embedding=embedding_model, persist_directory=chroma_db_dir)
    print("Vectorstore Chroma créé.")

retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
print(f"Retriever initialisé pour récupérer les {retriever.search_kwargs['k']} documents les plus pertinents.")


print("\n--- Étape 6 : Configuration du LLM (Gemini) ---")
print("Chargement du prompt RAG depuis Langchain Hub...")
prompt = hub.pull("rlm/rag-prompt")
print("Prompt chargé.")

print("Initialisation du LLM ChatGoogleGenerativeAI (Gemini-Pro)...")
try:
    llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.2)
    print("LLM Gemini-Pro initialisé avec succès.")
except Exception as e:
    print(f"ERREUR lors de l'initialisation du LLM ChatGoogleGenerativeAI : {e}")
    print("Assurez-vous que GOOGLE_API_KEY est correctement définie dans les secrets de Colab.")
    print("Vérifiez aussi que l'API Generative Language est activée pour votre projet Google Cloud.")
    exit("Processus arrêté car le LLM n'a pas pu être initialisé.")


print("\n--- Étape 7 : Construction de la Chaîne RAG ---")

def format_docs_with_sources(docs: List[Document]) -> str:
    formatted_content = ""
    unique_pages = set()

    for i, doc in enumerate(docs):
        formatted_content += f"Contenu source {i+1}:\n{doc.page_content}\n\n"
        if 'page' in doc.metadata:
            page_number = doc.metadata['page'] + 1
            unique_pages.add(str(page_number))

    if unique_pages:
        formatted_content += f"Sources des pages: {', '.join(sorted(list(unique_pages)))}\n"

    return formatted_content

# --- NOUVELLE DÉFINITION DE LA CHAÎNE RAG AVEC RUNNABLEPARALLEL ET RUNNABLELAMBDA ---
retrieval_chain = {"context": retriever, "question": RunnablePassthrough()}
generation_chain = prompt | llm | StrOutputParser()

rag_chain_with_sources = RunnableParallel(
    response=retrieval_chain | generation_chain,
    # Ici, nous enveloppons le lambda dans RunnableLambda
    source_documents=retrieval_chain | RunnableLambda(lambda x: x["context"])
).with_config(run_name="RAG Chain with Sources")

print("Chaîne RAG créée, prête à inclure les sources.")


print("\n--- Étape 8 : Tester la Chaîne RAG ---")

question = "Quelles sont les fonctions du code du travail? Et quelles sont les conditions de représentativité des organisations syndicales ?"
print(f"\nQuestion : {question}")

try:
    result = rag_chain_with_sources.invoke(question)

    print("\nRéponse Générée par Gemini :")
    print(result["response"])

    print("\nSources (extraits et métadonnées) :")
    print(format_docs_with_sources(result["source_documents"]))

except Exception as e:
    print(f"ERREUR lors de l'invocation de la chaîne RAG : {e}")
    print("Veuillez vérifier les étapes précédentes (authentification LLM, initialisation de Chroma).")



--- Étape 4 : Création des Embeddings ---
Initialisation du modèle d'embedding : all-MiniLM-L6-v2
Modèle d'embedding initialisé avec succès.

--- Étape 5 : Création ou chargement du Vectorstore Chroma ---
Chargement du vectorstore Chroma existant depuis : ./chroma_db_codetravail
Vectorstore Chroma chargé.
Retriever initialisé pour récupérer les 5 documents les plus pertinents.

--- Étape 6 : Configuration du LLM (Gemini) ---
Chargement du prompt RAG depuis Langchain Hub...
Prompt chargé.
Initialisation du LLM ChatGoogleGenerativeAI (Gemini-Pro)...
LLM Gemini-Pro initialisé avec succès.

--- Étape 7 : Construction de la Chaîne RAG ---
Chaîne RAG créée, prête à inclure les sources.

--- Étape 8 : Tester la Chaîne RAG ---

Question : Quelles sont les fonctions du code du travail? Et quelles sont les conditions de représentativité des organisations syndicales ?


ERREUR lors de l'invocation de la chaîne RAG : 404 models/gemini-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Veuillez vérifier les étapes précédentes (authentification LLM, initialisation de Chroma).


## UTILITE DU PROJET
1. **Qu'est-ce qu'une application RAG ? (Retrieval Augmented Generation)**
Imaginons une immense bibliothèque (dans notre cas, un PDF de 2775 pages sur le droit du travail) et qu'un utilisateur pose une question très spécifique.

**Sans RAG :** Si l'utilisateur posait cette question à un grand modèle de langage (LLM) comme Gemini sans RAG, il essayerait de répondre uniquement avec ce qu'il a appris pendant son entraînement. Le problème, c'est que Gemini n'a pas été spécifiquement entraîné sur le document de droit du travail. Il pourrait inventer des choses ("halluciner") ou donner des réponses génériques, car il ne "connaît" pas les détails du documents et donc du droit du travail.

**Avec RAG :** Le RAG résout ce problème en ajoutant une étape cruciale : _la recherche d'informations (Retrieval)_ avant la _génération de la réponse (Generation)_.

- **Recherche (Retrieval) :** Quand l'utilisateur poses une question, l'application RAG va d'abord chercher les passages les plus pertinents du document (corpus) qui sont susceptibles de contenir la réponse.
- **Augmentation (Augmented) :** Ces passages pertinents sont ensuite donnés au LLM (Gemini) en même temps que ta question. Le LLM est alors "augmenté" avec des connaissances spécifiques.
- **Génération (Generation) :** Fort de ces informations contextuelles, le LLM peut maintenant générer une réponse précise et basée sur les faits extraits de ton document.